In [ ]:
import numpy as np
import cv2
import random
np.set_printoptions(precision=2)

import ipywidgets.widgets as widgets
from IPython.display import display

In [ ]:
class MarkerTracker:
    a = 1
    b = 1.2

    def __init__(self,colour):
        self.marker_list = []
        self.conc_map = np.full(grid_size,0,dtype=np.float64)
        self.colour = colour
    
    def addMarker(self,pos):
        self.marker_list.append(pos)
        self.generateConcMap()

    def generateConcMap(self):
        conc_map = np.full(grid_size,0,dtype=np.float64)

        for i_y,i_x in np.ndindex(grid_size):
        # this iterates over every single cell in the grid
            conc = 0
            for marker in self.marker_list:
                # this iterates over the markers
                norm = 2 # 1 for taxicab distance, 2 for standard euclidean etc
                dist = (abs(marker[1] - i_y)**norm + abs(marker[0] - i_x)**norm)**(1/norm)
                
                dist += 1 # stops div zero errors

                conc += self.a/(dist**(self.b))

            conc_map[i_y,i_x] = conc

        self.conc_map=conc_map
    

Regarding the decision making & movement process of the walker; we shall follow some simple rules:

1. The walker will consider the 3 cells infront of it; and weight them based on the concentration
    1. Invalid moves (such as leaving the grid area, or into a wall) are given a zero weight
    2. If all three moves are invalid, the walker will turn all the way around and wait until next turn to move.

2. For each weight, we apply an exponent (`wander_exponent`) which controls how random or deterministic the walker is when picking from valid moves
    1. This is shown in an interactible format here: https://www.desmos.com/calculator/dqls0azfzl
    2. An exponent value of 0 means that all options have equal chance, regardless of concentration
    3. Higher values increase the influence of concentration where an infinite value would mean fully deterministic decision making
    4. Negative values make it run away from the marker

3. Once a decision has been made, the walker will store its new position and orientation; and all walkers will commit to their new state at the same time.
    1. This isnt very useful now, but is useful once we start introducing the action of depositing markers here during this process.



In [ ]:
class Walker:
    job_dict = {"scout":{"sens":"A","dep":"B"},
                "carrier":{"sens":"B","dep":"A"}}

    direction_lookup = [(-1,-1),
                        (0,-1),
                        (1,-1),
                        (1,0),
                        (1,1),
                        (0,1),
                        (-1,1),
                        (-1,0)]

    def __init__(self, pos, dir, job="scout"):
        self.pos = pos # tuple of 2 ints
        self.dir = dir # int between 0 and 7
        self.job = job # some key from job_dict

    def calculateMove(self):
        wander_exponent = 2 # this is the 

        left_dir = (self.dir-1) % 8
        right_dir = (self.dir+1) % 8

        left_pos = (self.pos[0]+self.direction_lookup[left_dir][0],self.pos[1]+self.direction_lookup[left_dir][1])
        front_pos = (self.pos[0]+self.direction_lookup[self.dir][0],self.pos[1]+self.direction_lookup[self.dir][1])
        right_pos = (self.pos[0]+self.direction_lookup[right_dir][0],self.pos[1]+self.direction_lookup[right_dir][1])

        """
        print("I am in position:",self.pos)
        print("the left is",left_pos)
        print("the front is",front_pos)
        print("the right is",right_pos)
        """
        conc_map = marker_lookup[self.job_dict[self.job]["sens"]].conc_map

        # these are unnormalised probabilities, divide by total to get actual probabilities
        if Walker.checkIfPosValid(left_pos):
            left_bias = conc_map[left_pos[1],left_pos[0]]**wander_exponent
        else:
            left_bias = 0

        if Walker.checkIfPosValid(front_pos):
            front_bias = conc_map[front_pos[1],front_pos[0]]**wander_exponent
        else:
            front_bias = 0

        if Walker.checkIfPosValid(right_pos):
            right_bias = conc_map[right_pos[1],right_pos[0]]**wander_exponent
        else:
            right_bias = 0

        if left_bias == 0 and front_bias == 0 and right_bias == 0:
            # this occurs when it rams into a wall, and all 3 of its choices are invalid
            # we will handle this by doing a 180, but not moving.

            self.new_pos = self.pos
            self.new_dir = (self.dir + 4) % 8
        else:
            # normal operation, at least one valid option
            choice = random.choices([(left_pos,left_dir),(front_pos,self.dir),(right_pos,right_dir)],weights=[left_bias,front_bias,right_bias])[0]
            self.new_pos = choice[0]
            self.new_dir = choice[1]

    def commitMove(self):
        self.pos = self.new_pos
        self.dir = self.new_dir

        del self.new_pos
        del self.new_dir

    def checkIfPosValid(pos):
        return (0<=pos[0]<grid_size[1]) and (0<=pos[1]<grid_size[0])


In [ ]:
grid_size = (5,29) # this is (y,x) format cos numpy
marker_lookup = {"A":MarkerTracker((0,255,0)),
                 "B":MarkerTracker((255,0,0))}

for i in range(2,15,3):
    marker_lookup["A"].addMarker((i,2))

for i in range(14,29,3):
    marker_lookup["B"].addMarker((i,2))

walker1 = Walker((0,1),4,"scout")
walker2 = Walker((0,3),2,"carrier")
walker_list = [walker1,walker2]

In [ ]:
cell_scale = 64
bottom_padding_size = 128
canvas_x_size = grid_size[1] * cell_scale
canvas_y_size = grid_size[0] * cell_scale + bottom_padding_size

display_markers = ["A","B"]

def arrow_cache_builder():
    # this is used to precalculate all 8 of the directional arrows that would have to be drawn
    arrow_cache = []
    angles = [-45,0,45,90,135,180,225,270]
    arrow_scale = 0.6
    original = np.asarray([[0,0],[1,-1]]) * (cell_scale/2) * arrow_scale
    for i,deg in enumerate(angles):
        rad = deg * (np.pi/180)
        c = np.cos(rad)
        s = np.sin(rad)
        rot_mat = np.asarray([[c,-s],[s,c]])

        transformed = np.matmul(rot_mat,original) + np.full((2,2),cell_scale/2)
        end = (transformed[0,0],transformed[1,0])
        tip = (transformed[0,1],transformed[1,1])
        arrow_cache += [{"tip":tip,"end":end}]

    return arrow_cache
    

def interp(vec1, vec2,t,clamp=False):
    if clamp:
        t = max(0,min(1,t))
    return vec1 * (1-t) + vec2 * t

def col_interp(col1,col2,t,clamp=False):
    vec1 = np.asarray(col1)
    vec2 = np.asarray(col2)
    res = interp(vec1,vec2,t,clamp)

    res = np.clip(res,0,255)
    col = (int(res[0]),int(res[1]),int(res[2]))
    return col

def blend_colour(col_list):
    # this is used to blend the colours of the different markers when their fields intersect
    a = 2
    total = [0,0,0]
    for col in col_list:
        total[0] += col[0]**a
        total[1] += col[1]**a
        total[2] += col[2]**a

    total = (int((total[0]/len(col_list))**(1/a)),int((total[1]/len(col_list))**(1/a)),int((total[2]/len(col_list))**(1/a)))
    return total

def drawCells(canvas):
    # this part of the drawing procedure plots all of the cells to the canvas, as well as the concentration/colour of the markers
    base_cell_bg_colour = (128,128,128)

    for i_y,i_x in np.ndindex(grid_size):
        cell_tl = (int(i_x*cell_scale),int(i_y*cell_scale))
        cell_br = (cell_tl[0] + cell_scale-1,cell_tl[1] + cell_scale-1)

        col_list = []
        if(len(display_markers) != 0):
            for label in display_markers:
                #iterate through the labels of all the markers we're displaying
                marker_tracker = marker_lookup[label]

                # this is temporary; it needs to generate a colour based on the concentration and base colours of all of the markers we're trying to display
                conc=marker_tracker.conc_map[i_y,i_x]
                temp_col = col_interp(base_cell_bg_colour,marker_tracker.colour,conc,False) # a 'False' flag here means that it will 'overdrive' the colours for values greater than 1
                col_list += [temp_col]
            
            cell_bg_colour = blend_colour(col_list)
        else:
            cell_bg_colour = base_cell_bg_colour
        canvas = cv2.rectangle(canvas, cell_tl, cell_br, cell_bg_colour, -1)
        cell_edge_colour = col_interp((0,0,0),cell_bg_colour,0.6,True)
        canvas = cv2.rectangle(canvas, cell_tl, cell_br, cell_edge_colour, thickness=int(cell_scale * 0.05))

    return canvas

def drawWalkers(canvas):
    # pretty self explanatory, it draws all the walkers
    colour_dict = {"scout":(0,0,0),
                   "carrier":(0,0,200)}
    for walker in walker_list:
        pos = walker.pos
        dir = walker.dir
        colour = colour_dict[walker.job]

        temp = arrow_cache[dir]
        tip = temp["tip"]
        end = temp["end"]

        tip_x = int(pos[0]*cell_scale + tip[0])
        tip_y = int(pos[1]*cell_scale + tip[1])
        end_x = int(pos[0]*cell_scale + end[0])
        end_y = int(pos[1]*cell_scale + end[1])

        canvas = cv2.arrowedLine(canvas, (end_x,end_y), (tip_x,tip_y), colour, thickness=int(cell_scale * 0.05), tipLength = 0.2) 
    return canvas

arrow_cache = arrow_cache_builder()

def draw():
    canvas = np.full((int(canvas_y_size),int(canvas_x_size),3),150)
    canvas = drawCells(canvas)
    canvas = drawWalkers(canvas)
    return canvas

canvas = draw()

In [ ]:
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg',value)[1])

image_widget = widgets.Image(format='jpeg', width=512, height=512)
image_widget.value = bgr8_to_jpeg(canvas)
display(image_widget)

In [ ]:
print(marker_lookup["A"].conc_map)

In [ ]:
import time
canvas= draw()
image_widget.value = bgr8_to_jpeg(canvas)
time.sleep(2)

update_rate = 0.2
c = 0
while True:
    start = time.time()
    c+=update_rate


    for walker in walker_list:
        if c > 30:
            if walker.job=="scout":
                walker.job="carrier"
            else:
                walker.job="scout"
        walker.calculateMove()
    for walker in walker_list:
        walker.commitMove()

    if c>30:
        print("swapping roles!")
        c=0
    
    canvas= draw()
    image_widget.value = bgr8_to_jpeg(canvas)
    end=time.time()
    delta = end-start
    if(delta<update_rate):
        time.sleep(update_rate-delta)